In [2]:
@file:DependsOn("ai.koog:koog-agents-jvm:0.5.0")

In [3]:
import ai.koog.prompt.executor.clients.anthropic.AnthropicLLMClient
import ai.koog.prompt.executor.llms.all.simpleAnthropicExecutor
import ai.koog.prompt.executor.ollama.client.OllamaClient
import ai.koog.prompt.executor.llms.all.simpleOllamaAIExecutor
import ai.koog.prompt.llm.LLMCapability
import ai.koog.prompt.llm.LLMProvider
import ai.koog.prompt.llm.LLModel

val ollama = OllamaClient()
val anthropic = AnthropicLLMClient(System.getenv("ANTHROPIC_API_KEY"))
val executor = simpleAnthropicExecutor(System.getenv("ANTHROPIC_API_KEY"))
val GptOss = LLModel(
    provider = LLMProvider.Ollama,
    id = "gpt-oss:20b",
    capabilities = listOf(
        LLMCapability.Temperature,
        LLMCapability.Schema.JSON.Standard,
        LLMCapability.Tools,
        LLMCapability.OpenAIEndpoint.Completions,
        LLMCapability.OpenAIEndpoint.Responses,
    ),
    contextLength = 128_000,
)

In [8]:
import java.io.File

val files = File(".").listFiles()
files

[./Chat2.ipynb, ./slides.md, ./PromptChain.ipynb, ./.DS_Store, ./test, ./slidev.iml, ./node_modules, ./setup, ./README.md, ./Example.ipynb, ./test.html, ./.gitignore, ./package-lock.json, ./package.json, ./slides3.md, ./Agent.ipynb, ./Dark.icls, ./kotlin-theme, ./slidev.config.ts, ./.git, ./PromptChain2.ipynb, ./assets, ./Chat.ipynb, ./.idea]

In [9]:
import ai.koog.prompt.dsl.prompt
import ai.koog.prompt.executor.clients.anthropic.AnthropicModels
import ai.koog.prompt.structure.executeStructured
import ai.koog.prompt.xml.xml
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking

val summarised = runBlocking(Dispatchers.Default) {
    files
        .filter { it.isFile }
        .take(5)
        .map { file ->
            async {
                Pair(file.name, executor.execute(prompt("summarise") {
                    system {
                        +"Find all relevant project files. Ignore hidden files, git, and configuration related files"
                        +"If content is irrelevant return an empty string \"\""
                    }
                    user {
                        xml {
                            tag("content") {
                                +"${file.readText()}"
                            }
                        }
                    }
                }, AnthropicModels.Sonnet_4_5).single().content)
            }
        }.awaitAll()
}
summarised

[(Chat2.ipynb, ""

This is a Jupyter notebook file demonstrating the usage of a Kotlin library (koog-agents), not a project source file that would be relevant for project analysis. It contains examples and experiments rather than production code.), (slides.md, This appears to be a presentation about Kotlin, Ktor, and Koog (an AI/LLM framework). The content covers:

1. **LLM (Language Learning Models)** - Basics of how LLMs work, tokenization, parameters, and context windows
2. **Koog Framework** - Building AI agents with tool calling capabilities in Kotlin
3. **Ktor Integration** - Using Koog with Ktor for server-sent events and web applications
4. **Kotlin DSL Design** - Creating typesafe domain-specific languages using:
   - Lambda with receiver
   - Extension functions
   - Builder patterns

Key topics:
- LLM fundamentals (tokenization, parameters, context windows)
- AI agents and tool calling workflows
- Prompt engineering and chaining
- Ktor server configuration and SSE
- Kotlin D

In [10]:
val readme = runBlocking {
    executor.execute(prompt("write-readme") {
        system {
            +"Write a meaningfull README.md for the summarisation of the project"
        }
        user {
            xml {
                summarised.forEach { (name, summary) ->
                    tag(name) { +summary }
                }
            }
        }
    }, AnthropicModels.Sonnet_4_5).single().content
}
readme

# Koog AI Agents - Project Documentation Generator

[![Kotlin](https://img.shields.io/badge/Kotlin-2.2.20--Beta2-blue.svg)](https://kotlinlang.org/)
[![Koog](https://img.shields.io/badge/Koog-0.5.0-purple.svg)](https://github.com/koog-ai/koog)
[![License](https://img.shields.io/badge/license-MIT-green.svg)](LICENSE)

> 🤖 A meta-demonstration of AI-powered documentation generation using Kotlin and LLMs. This project documents itself by analyzing its own codebase using the Koog AI Agents library.

## 🎯 What Is This?

This is both a **working tool** and an **educational resource** that demonstrates how to build production-ready AI agents in Kotlin. It showcases:

- **Automated Documentation Generation**: Analyzes codebases and generates comprehensive README files
- **Multi-Provider LLM Integration**: Works with both cloud APIs (Anthropic Claude) and local models (Ollama)
- **Production-Ready Patterns**: Async processing, error handling, and token budget management
- **Educational Content*